In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
! pip install rasterio 1>/dev/null

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/sen2dwater/USANNIOMIT')
# sys.path.append('/content/drive/MyDrive/SEN2DWATER')

! pip install importlib 1>/dev/null
import importlib

MessageError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from dataio.datahandler import datahandler
from dataio.datareader import datareader

from dataio.datahandler import datahandler
from dataio.datareader import datareader
# from models.LSTMv1 import LSTMv1
# from models.BLSTMv1 import BLSTMv1
from models.TDCNNv1 import TDCNNv1

# from utils.plot_dataset import plot_series 

from config import *

import matplotlib.pyplot as plt
import os

%matplotlib inline

In [ ]:
#======================================= Loading the dataset ========================================
# dataset_root_path = '/content/drive/MyDrive/sen2dwater/USANNIOMIT/DATASET_2016_2022'
dataset_root_path = '/content/drive/MyDrive/SEN2DWATER'

dh       = datahandler(dataset_root_path)
keys     = list(dh.paths.keys())
t_len    = len(dh.paths[keys[0]])

print('{:=^100}'.format(' Loading the dataset '))
print('\t -{:<50s} {}'.format('Number of GeoLocation', len(keys)))
print('\t -{:<50s} {}'.format('Number of Images per GeoLocation', t_len))

#========================================== Split dataset ===========================================
train_set, val_set = dh.split(SPLIT_FACTOR)
print('{:=^100}'.format(' Splitting the dataset '))
print('\t -{:<50s} {}'.format('Number of GeoLocation (training)', len(train_set.keys())))
print('\t -{:<50s} {}'.format('Number of GeoLocation (validation)', len(val_set.keys())))

tdcnnv1 = TDCNNv1(shape=(T_LEN, PATCH_WIDTH, PATCH_HEIGHT, BANDS))
print('{:=^100}'.format(' Building the Model '))
print(tdcnnv1.model.summary())
#========================================== Train model =============================================
print('{:=^100}'.format(' Training the Model '))
# history = tdcnnv1.train(train_set, val_set, normalize = True)

tdcnnv1.generateData(train_set, val_set)

In [ ]:
# history = tdcnnv1.train(train_set, val_set, normalize = True)
# tdcnnv1.model.save('/content/drive/MyDrive/sen2dwater/USANNIOMIT/tdcnnv1.model.h5')

ndwi_history, ndwi_model = tdcnnv1.train('NDWI', train_set, val_set, normalize = True)
ndwi_model.save('/content/drive/MyDrive/sen2dwater/USANNIOMIT/ndwi_model.h5')

ndvi_history, ndvi_model = tdcnnv1.train('NDVI', train_set, val_set, normalize = True)
ndvi_model.save('/content/drive/MyDrive/sen2dwater/USANNIOMIT/ndvi_model.h5')

nddi_history, nddi_model = tdcnnv1.train('NDDI', train_set, val_set, normalize = True)
nddi_model.save('/content/drive/MyDrive/sen2dwater/USANNIOMIT/nddi_model.h5')

In [ ]:
LEN_SERIES = t_len
fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (4*LEN_SERIES, 4))

axes[0].plot(tdcnnv1.model.history.loss, label = 'training loss')
axes[0].plot(tdcnnv1.model.history['val_loss'], label = 'validation loss')
axes[0].set_title('Huber Loss')
axes[0].legend()

axes[1].plot(tdcnnv1.model.history['mae'], label = 'training MAE')
axes[1].plot(tdcnnv1.model.history['val_mae'], label = 'validation MAE')
axes[1].set_title('MAE')
axes[1].legend()

axes[2].plot(tdcnnv1.model.history['mse'], label = 'training MSE')
axes[2].plot(tdcnnv1.model.history['val_mse'], label = 'validation MSE')
axes[2].set_title('MSE')
axes[2].legend()
plt.savefig(name + 'Training.pdf')
plt.show()